## Download

In [9]:
import requests
import pathlib
import patoolib
import glob
import os
import subprocess

def download_file(url, dir, name):
    local_filename = dir / name
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                # If you have chunk encoded response uncomment if
                # and set chunk_size parameter to None.
                #if chunk: 
                f.write(chunk)
    return local_filename


In [14]:
from tqdm.notebook import tqdm

# 10 - 30
ids = ["4264835", "4264834", "4264838", "4264836", "4264839", "4266983", "4266984", "4266987", "4266973", "4267013", "4266962", "4267008", "4267023", "4267010", "4264284", "4266985", "4267040", "4267038", "4266966", "4266997", "4266963", "4266976", "4264837", "4264833", "4267026", "4267005", "4267006", "4266982", "4264297", "4266992", "4266972", "4266993", "4266970", "4267022", "4267033", "4267018", "4267000", "4267009", "4267012", "4267007", "4267047", "4266969", "4266967", "4267003", "4266995", "4267039", "4266975", "4267017", "4266965", "4266994", "4267019", "4266986", "4267043", "4267020", "4267001", "4267032", "4266960", "4266996", "4266978", "4267025", "4267004", "4266998", "4266990", "4267016", "4267041", "4267002", "4266961", "4266977", "4266980", "4266974", "4267028", "4266968", "4267021", "4267015", "4266964", "4267045", "4267049", "4266979", "4266989", "4267035", "4267030", "4266988", "4267024", "4266981", "4267510", "4267599", "4267550", "4267523", "4267581", "4267553", "4267552", "4267535", "4267589", "4267571", "4267588", "4267540", "4267604", "4267558", "4267529", "4267537", "4267606", "4267578", "4267610", "4267598", "4267545", "4267506", "4267505", "4267572", "4267605", "4267576", "4267541", "4267508", "4267527", "4267538", "4267500", "4267515", "4267607", "4267514", "4267539", "4267517", "4267513", "4267582", "4267609", "4267562", "4267591", "4267556", "4267518", "4267597", "4267528", "4267509", "4267548", "4267544", "4267512", "4267601", "4267611", "4267593", "4267586", "4267551", "4267546", "4267499", "4267543", "4267616", "4267568", "4267564", "4267579", "4267520", "4267554", "4267613", "4267557", "4267519", "4267561", "4267522", "4267503", "4267603", "4267596", "4267531", "4267566", "4267577", "4267615", "4267521", "4267612", "4267504", "4267570", "4267569", "4267532", "4267595", "4267600", "4267574", "4267585", "4267498", "4267507", "4267516", "4267497", "4267573", "4267533", "4267602", "4267549", "4267565", "4267608", "4267575", "4267614", "4267563", "4267526", "4267542", "4267525", "4267524", "4267536", "4267590", "4267592", "4267584", "4267511", "4267580", "4267502", "4267559", "4267534", "4267547", "4267501", "4267560", "4267555", "4267587", "4267567", "4267594", "4267583", "4267530", "4274659", "4274650", "4274707", "4274658", "4274627", "4274634", "4274668", "4274629", "4274642", "4274694", "4274701", "4274683", "4274706", "4274672", "4274647", "4274675", "4274646", "4274631", "4274687", "4274665", "4274684", "4274612", "4274638", "4274685", "4274624", "4274618", "4274667", "4274653", "4274625", "4274664", "4274628", "4274692", "4274700", "4274623", "4274686", "4274699", "4274671", "4274679", "4274669", "4274705", "4274663", "4274662", "4274678", "4274702", "4274613", "4274635", "4274680", "4274690", "4274622", "4274626", "4274695", "4274666", "4274617", "4274641", "4274637", "4274709", "4274619", "4274616", "4274681", "4274632", "4274698", "4274677", "4274688", "4274656", "4274640", "4274696", "4274703", "4274645", "4274697", "4274615", "4274661", "4274611", "4274655", "4274674", "4274693", "4274708", "4274636", "4274691", "4274649", "4274660", "4274614", "4274682", "4274644", "4274673", "4274652", "4274710", "4274651", "4274676", "4274633", "4274654", "4274639", "4274648", "4274621", "4274643", "4274657", "4274689", "4274704", "4274620", "4274670", "4274630", "4275267", "4275269", "4275161", "4275263", "4275203", "4275188", "4275176", "4275238", "4275258", "4275253", "4275182", "4275187", "4275177", "4275216", "4275268", "4275205", "4275195", "4275165", "4275225", "4275181", "4275169", "4275209", "4275172", "4275266", "4275200", "4275192", "4275265", "4275167", "4275261", "4275194", "4275264", "4275256", "4275255", "4275186", "4275198", "4275248", "4275220", "4275245", "4275232", "4275247", "4275189", "4275173", "4275241", "4275210", "4275201", "4275207", "4275164", "4275227", "4275190", "4275254", "4275219", "4275234", "4275174", "4275183", "4275257", "4275224", "4275239", "4275179", "4275197", "4275191", "4275259", "4275252", "4275171", "4275260", "4275270", "4275262", "4275211", "4275180", "4275208", "4275235", "4275193", "4275233", "4275162", "4275178", "4275175", "4275243", "4275221", "4275163", "4275230", "4275199", "4275170", "4275185", "4275222", "4275231", "4275236", "4275184", "4275206", "4275212", "4275213", "4275223", "4275242", "4275202", "4275214", "4275250", "4275246", "4275237", "4275249", "4275226", "4275218", "4275168", "4275215", "4275217", "4275251", "4275204", "4275240", "4275166", "4275228", "4275229", "4275196", "4275244", "4275630", "4275638", "4275617", "4275649", "4275633", "4275601", "4275629", "4275604", "4275632", "4275631", "4275644", "4275652", "4275685", "4275653", "4275654", "4275636", "4275594", "4275620", "4275660", "4275602", "4275616", "4275665", "4275663", "4275650", "4275694", "4275679", "4275622", "4275690", "4275599", "4275670", "4275668", "4275595", "4275669", "4275640", "4275608", "4275610", "4275642", "4275691", "4275700", "4275687", "4275611", "4275699", "4275673", "4275692", "4275667", "4275677", "4275686", "4275697", "4275695", "4275656", "4275607", "4275639", "4275614", "4275671", "4275666", "4275648", "4275596", "4275664", "4275662", "4275651", "4275646", "4275625", "4275645", "4275603", "4275593", "4275659", "4275681", "4275634", "4275678", "4275643", "4275657", "4275623", "4275647", "4275598", "4275658", "4275606", "4275680", "4275676", "4275696", "4275618", "4275609", "4275682", "4275592", "4275693", "4275619", "4275637", "4275661", "4275672", "4275701", "4275689", "4275627", "4275688", "4275597", "4275635", "4275624", "4275683", "4275626", "4275605", "4275613", "4275655", "4275674", "4275698", "4275612", "4275621", "4275641", "4275615", "4275600", "4275628", "4275675", "4275684", "4281186", "4281202", "4281206", "4281193", "4281223", "4281191", "4281200", "4281195", "4281194", "4281214", "4281185", "4281189", "4281187", "4281221", "4281208", "4281219", "4281211", "4281207", "4281203", "4281212", "4281197", "4281218", "4281198", "4281199", "4281224", "4281215", "4281190", "4281196", "4281210", "4281216", "4281220", "4281201", "4281192", "4281188", "4281204", "4281222", "4281213", "4281217", "4281209", "4281205", "4285078", "4285087", "4285072", "4285075", "4285077", "4285080", "4285086", "4285076", "4285071", "4285083", "4285084", "4285090", "4285082", "4285089", "4285079", "4285085", "4285088", "4285073", "4285081", "4285074", "4291669", "4291671", "4291635", "4291640", "4291666", "4291614", "4291668", "4291625", "4291670", "4291643", "4291659", "4291675", "4291612", "4291660", "4291639", "4291641", "4291649", "4291653", "4291622", "4291672", "4291654", "4291624", "4291674", "4291616", "4291630", "4291618", "4291623", "4291628", "4291631", "4291652", "4291677", "4291619", "4291665", "4291610", "4291617", "4291636", "4291661", "4291634", "4291627", "4291676", "4291645", "4291642", "4291644", "4291656", "4291638", "4291657", "4291648", "4291662", "4291667", "4291629", "4291650", "4291658", "4291621", "4291637", "4291633", "4291651", "4291673", "4291664", "4291620", "4291611", "4291663", "4291647", "4291646", "4291626", "4291613", "4291632", "4291655", "4291615", "4302496", "4302520", "4302505", "4302502", "4302501", "4302508", "4302490", "4302512", "4302497", "4302522", "4302503", "4302491", "4302504", "4302509", "4302518", "4302493", "4302521", "4302489", "4302499", "4302514", "4302517", "4302516", "4302515", "4302495", "4302511", "4302494", "4302523", "4302500", "4302519", "4302492", "4302507", "4302506", "4302513", "4302510", "4302498", "4378241", "4378237", "4378240", "4378238", "4378239", "4378243", "4378245", "4378247", "4378246", "4378244", "4378242", "4378248", "4378249", "4382656", "4382705", "4382708", "4382702", "4382653", "4382665", "4382652", "4382668", "4382673", "4382659", "4382651", "4382667", "4382650", "4382657", "4382662", "4382661", "4382658", "4382671", "4382660", "4382669", "4382654", "4382670", "4382663", "4382666", "4382664", "4382674", "4382672", "4382655", "4382693", "4382703", "4382704", "4382694", "4382695", "4382711", "4382690", "4382707", "4382701", "4382692", "4382697", "4382709", "4382710", "4382691", "4382698", "4382699", "4382696", "4382706", "4382689", "4382700", "4383064", "4383069", "4383086", "4383081", "4383068", "4383076", "4383065", "4383071", "4383062", "4383061", "4383070", "4383085", "4383073", "4383063", "4383074", "4383066", "4383079", "4383087", "4383078", "4383084", "4383072", "4383082", "4383060", "4383080", "4383059", "4383083", "4383067", "4383075", "4383058", "4383077", "4402471", "4402473", "4402475", "4402479", "4402462", "4402472", "4402480", "4402478", "4402468", "4402474", "4402467", "4402481", "4402476", "4402466", "4402464", "4402477", "4402470", "4402469", "4402463", "4402465", "4404140", "4404141", "4404142", "4404215", "4404221", "4404211", "4404216", "4404212", "4404220", "4404218", "4404214", "4404225", "4404219", "4404226", "4404217", "4404222", "4404213", "4404224", "4404223", "4404210", "4404297", "4404295", "4404288", "4404301", "4404298", "4404302", "4404294", "4404286", "4404287", "4404296", "4404299", "4404303", "4404293", "4404292", "4404289", "4404290", "4404291", "4404300", "4404304", "4411736", "4411734", "4411735", "4411737", "4411741", "4411740", "4411739", "4411738", "4411793", "4411795", "4411472", "4411471", "4411470", "4411469", "4411475", "4411476", "4411482", "4411558", "4411559", "4411557", "4411796", "4411560", "4411563", "4411562", "4411561", "4411564", "4411565", "4411566", "4411794", "4411797", "4411407", "4411408", "4411406", "4411405", "4411417", "4411414", "4411415", "4411416", "4411420", "4411419", "4410885", "4410886", "4411396", "4411397", "4411398", "4411395", "4411402", "4411401", "4411403", "4411404", "4411426", "4411425", "4411424", "4411423", "4411428", "4411430", "4411427", "4411429", "4411433", "4411432", "4410826", "4410824", "4410825", "4410829", "4410828", "4410827", "4410832", "4410830", "4410833", "4410831", "4410836", "4410837", "4410835", "4410834", "4410838", "4410840", "4410859", "4410856", "4410857", "4410858", "4409574", "4409575", "4409573", "4410087", "4410088", "4410086", "4410401", "4410402", "4410400", "4410839", "4404145", "4404449", "4404146", "4404150", "4404148", "4404151", "4404149", "4404366", "4404147", "4404367", "4404143", "4404390", "4404389", "4404399", "4404452", "4404451", "4404450", "4404448", "4404447", 
]

for id in tqdm(ids):
    url = f"https://dataverse.harvard.edu/api/access/datafile/{id}"
    name = url.split("/")[-1]
    path = "./data/" + name
    if os.path.exists(path):
        continue
    print(f"downloading {name}")
    download_file(url=url, dir=pathlib.Path("./downloaded"),
                  name=name + ".rar")
    print(f"extracting {name}")
    if not os.path.exists(path):
        os.mkdir(path)
    # patoolib.extract_archive(pathlib.Path("./downloaded") 
    #                          / (name + ".rar"),
    #                          outdir="./data/" + name)

    subprocess.run(["7z", "x", pathlib.Path("./downloaded") / (name + ".rar"), f"-o./data/{name}"])
    os.rename(glob.glob("./data/" + name + "/*")[0], "./data/" + name + "/ct")
    os.remove(pathlib.Path("./downloaded") / (name + ".rar"))

  0%|          | 0/946 [00:00<?, ?it/s]

downloading 4264835
extracting 4264835

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,24 CPUs Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz (50654),ASM,AES-NI)

Scanning the drive for archives:
1 file, 114352364 bytes (110 MiB)

Extracting archive: downloaded/4264835.rar
--
Path = downloaded/4264835.rar
Type = Rar
Physical Size = 114352364
Solid = -
Blocks = 429
Multivolume = -
Volumes = 1

Everything is Ok

Folders: 2
Files: 427
Size:       224509972
Compressed: 114352364
downloading 4264834
extracting 4264834

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,24 CPUs Intel(R) Xeon(R) Gold 6126 CPU @ 2.60GHz (50654),ASM,AES-NI)

Scanning the drive for archives:
1 file, 120873672 bytes (116 MiB)

Extracting archive: downloaded/4264834.rar
--
Path = downloaded/4264834.rar
Type = Rar
Physical Size = 120873672
Solid 

In [ ]:
%slack

In [20]:
import glob
from tqdm.notebook import tqdm

for idx, file in enumerate(tqdm(glob.glob("./data/*"))):
    if '__' in file:
        continue
    # os.remove(file + "/drr.pickle")
    # os.remove(file + "/fbp.pickle")
    # print(file)
    if len(glob.glob(file + "/ct/*")) == 1:
        for folder in glob.glob(file + "/ct/*"):
            # print(folder)
            for img in glob.glob(folder + "/*.dcm"):
                # print(img)
                imgname = img.split("/")[-1]
                # print(file + "/ct/" + imgname)
                os.rename(img, file + "/ct/" + imgname)
            for img in glob.glob(folder + "/*"):
                if not ".dcm" in img:
                    # print(img)
                    imgname = img.split("/")[-1]
                    # print(file + "/ct/" + imgname + ".dcm")
                    os.rename(img, file + "/ct/" + imgname + ".dcm")
    if len(glob.glob(file + "/ct")) == 0 or not os.path.isdir(file + "/ct"):
        os.remove(file + "/ct")
        os.mkdir(file + "/ct")
        for img in glob.glob(file + "/*.dcm"):
            imgname = img.split("/")[-1]
            # print(img)
            # print(file + "/ct/" + imgname)
            os.rename(img, file + "/ct/" + imgname)
        # print(file)

  0%|          | 0/951 [00:00<?, ?it/s]

## DRR, FBP, Resize

In [1]:
import importlib
import imageio.v2 as imageio
from patient import patient
import glob

# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/6ACUZJ

from tqdm.notebook import tqdm
for vol_path in tqdm(glob.glob("./data/*")):
    if "__" in vol_path:
        continue
    vol = vol_path.split("/")[-1]
    # print(vol)
    p = patient.patient(name=vol)

# imageio.mimsave(f"pics/drr.gif", p.drr.img)
%slack

  0%|          | 0/188 [00:00<?, ?it/s]

DRR:   0%|          | 0/450 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.51 GiB (GPU 0; 11.76 GiB total capacity; 7.30 GiB already allocated; 1.50 GiB free; 8.19 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import imageio.v2 as imageio
import cupy as cp
from patient import patient

p = patient.patient(name="4410088")

# p.generate_drr(cont=False)

# p.generate_drr(cont=False)
# p.generate_posdrr(cont=False)

for idx, img in enumerate(p.drr.img):
    new_img = img
    new_img[new_img > 255] = 255
    p.drr.img[idx] = new_img.astype("uint8")
    # print(cp.max(cp.ravel(p.drr.img[idx])), cp.min(cp.ravel(p.drr.img[idx])))

p.generate_fbp(load_all=False)

imgs = [cp.asnumpy(img) for img in p.drr.img]
imageio.mimsave("drr.gif", imgs)

imgs = [cp.asnumpy(img) for img in p.posdrr.img]
imageio.mimsave("posdrr.gif", imgs)

FBP:   0%|          | 0/400 [00:00<?, ?it/s]

TypeError: patient.generate_fbp() got an unexpected keyword argument 'cont'

In [ ]:
a = 200
imageio.imsave("test3.png", cp.asnumpy(p.get_equiv_fbp(a)))
imageio.imsave("test4.png", cp.asnumpy(p.ct.img[a]))
%slack

In [ ]:
import glob, os, datetime

for vol in glob.glob("./data/*"):
    for file in glob.glob(vol + "/*"):
        if ".pickle" in file:
            os.remove(file)
            # print(file)

## Pix2Pix Prepare

In [6]:
from pix2pix.datasets import combine_ct_and_fbp
combine_ct_and_fbp.prep("./pix2pix/datasets/ctfbp")
%slack


  0%|          | 0/131 [00:00<?, ?it/s]

4410857
4266993
4267031
4410835
4267032
4411407
4410858
4410859
4411416
4410828
4410837
4267027


4267027:   0%|          | 0/90 [00:00<?, ?it/s]

4410838
4404366
4411557
4410402
4409575
4410827
4411429
4411469
4411414
4411471
4266990
4266973
4266978
4267000


4267000:   0%|          | 0/90 [00:00<?, ?it/s]

...

4267022:   0%|          | 0/90 [00:00<?, ?it/s]

4410886


4410886:   0%|          | 0/90 [00:00<?, ?it/s]

4411428


4411428:   0%|          | 0/90 [00:00<?, ?it/s]